<a href="https://colab.research.google.com/github/alarcon7a/paligemma-tutorial/blob/main/PaliGemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cargar librerias

In [ ]:
!pip install -q -U accelerate bitsandbytes git+https://github.com/huggingface/transformers.git gradio

### HuggingFace Login

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Cargando modelo

In [ ]:
from transformers import AutoTokenizer, PaliGemmaForConditionalGeneration, PaliGemmaProcessor
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "google/paligemma-3b-mix-224"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16)
processor = PaliGemmaProcessor.from_pretrained(model_id)

In [ ]:
import torch
import numpy as np
from PIL import Image
import requests

input_image = Image.open('/content/animals.png')

In [ ]:
input_image

## Describir imagenes

In [ ]:
input_text = "caption"
inputs = processor(text=input_text, images=input_image,
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
model.to(device)
inputs = inputs.to(dtype=model.dtype)

with torch.no_grad():
  output = model.generate(**inputs, max_length=496)

result = processor.decode(output[0], skip_special_tokens=True)
print(result)

In [ ]:
input_text = "caption es"
inputs = processor(text=input_text, images=input_image,
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
model.to(device)
inputs = inputs.to(dtype=model.dtype)

with torch.no_grad():
  output = model.generate(**inputs, max_length=496)

result = processor.decode(output[0], skip_special_tokens=True)
print(result)

## Q&A en imagenes


In [ ]:
input_text = "cuantos gatos hay?"
inputs = processor(text=input_text, images=input_image,
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
model.to(device)
inputs = inputs.to(dtype=model.dtype)

with torch.no_grad():
  output = model.generate(**inputs, max_length=496)

result = processor.decode(output[0], skip_special_tokens=True)
print(result)

In [ ]:
input_text = "de que raza son los perros?"
inputs = processor(text=input_text, images=input_image,
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
model.to(device)
inputs = inputs.to(dtype=model.dtype)

with torch.no_grad():
  output = model.generate(**inputs, max_length=496)

result = processor.decode(output[0], skip_special_tokens=True)
print(result)

In [ ]:
input_text = "de que raza es el gato?"
inputs = processor(text=input_text, images=input_image,
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
model.to(device)
inputs = inputs.to(dtype=model.dtype)

with torch.no_grad():
  output = model.generate(**inputs, max_length=496)

result = processor.decode(output[0], skip_special_tokens=True)
print(result)

## Deteccion de objetos

In [ ]:
input_text = "detect golden retriever"
inputs = processor(text=input_text, images=input_image,
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
model.to(device)
inputs = inputs.to(dtype=model.dtype)

with torch.no_grad():
  output = model.generate(**inputs, max_length=496)

result = processor.decode(output[0], skip_special_tokens=True)
print(result)

### Limpiando las coordenadas

In [ ]:
output = result[len(input_text):].lstrip("\n")
output_list = output.split(';')

In [ ]:
output_list

### Generar coordenadas

In [ ]:
import re

image_width, image_height = input_image.size

_SEGMENT_DETECT_RE = re.compile(
    r'(.*?)' +
    r'<loc(\d{4})>' * 4 + r'\s*' +
    '(?:%s)?' % (r'<seg(\d{3})>' * 16) +
    r'\s*([^;<>]+)? ?(?:; )?',
)

labels, boxes = [], []

for _ in output_list:
    m = _SEGMENT_DETECT_RE.match(_)
    gs = list(m.groups())
    before = gs.pop(0)
    name = gs.pop()
    y1, x1, y2, x2 = [int(x) / 1024 for x in gs[:4]]
    y1 = y1  * image_height
    x1 = x1  * image_width
    y2 = y2  * image_height
    x2 = x2  * image_width
    boxes.append((x1, y1, x2, y2))
    labels.append(name)


In [ ]:
boxes

In [ ]:
labels

### Generando imagen

In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
image_copy = input_image.copy()

draw = ImageDraw.Draw(image_copy)
for i in range(len(boxes)):
  draw.rectangle(boxes[i], outline="red", width=8)
  plt.text(boxes[i][0], boxes[i][1], labels[i], color='red', fontsize=12)
# Mostrar la imagen con el rectángulo
plt.imshow(image_copy)
plt.axis('off')
plt.show()

### Un ejemplo mas

In [ ]:
input_image = Image.open('/content/bodegon.png')

In [ ]:
input_image

In [ ]:
input_text = "detect bottle ; apple; lemon"
inputs = processor(text=input_text, images=input_image,
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
model.to(device)
inputs = inputs.to(dtype=model.dtype)

with torch.no_grad():
  output = model.generate(**inputs, max_length=496)

result = processor.decode(output[0], skip_special_tokens=True)
print(result)

In [ ]:
output = result[len(input_text):].lstrip("\n")
output_list = output.split(';')
output_list

In [ ]:
def get_coordinates(output_list, image_height,image_width ):
  labels, boxes = [], []

  for _ in output_list:
      m = _SEGMENT_DETECT_RE.match(_)
      gs = list(m.groups())
      before = gs.pop(0)
      name = gs.pop()
      y1, x1, y2, x2 = [int(x) / 1024 for x in gs[:4]]
      y1 = y1  * image_height
      x1 = x1  * image_width
      y2 = y2  * image_height
      x2 = x2  * image_width
      boxes.append((x1, y1, x2, y2))
      labels.append(name)
  return boxes, labels

In [ ]:
image_width, image_height = input_image.size
boxes, labels = get_coordinates(output_list, image_height,image_width)

In [ ]:
def draw_detection(image, boxes, labels):
  draw = ImageDraw.Draw(image_copy)
  for i in range(len(boxes)):
    draw.rectangle(boxes[i], outline="red", width=8)
    plt.text(boxes[i][0], boxes[i][1], labels[i], color='red', fontsize=12)
  # Mostrar la imagen con el rectángulo
  plt.imshow(image_copy)
  plt.axis('off')
  plt.show

In [ ]:
image_copy = input_image.copy()
draw_detection(image_copy,boxes,labels)

## Segmentacion de objetos

In [ ]:
input_text = "segment bottle ; lemon"
inputs = processor(text=input_text, images=input_image,
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
model.to(device)
inputs = inputs.to(dtype=model.dtype)

with torch.no_grad():
  output = model.generate(**inputs, max_length=496)

result = processor.decode(output[0], skip_special_tokens=True)
print(result)

In [ ]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="google/paligemma-hf", filename="vae-oid.npz",
                repo_type="space", local_dir="./")

In [ ]:
import re
import flax.linen as nn
import jax
import jax.numpy as jnp
import numpy as np
import functools
import PIL

_MODEL_PATH = 'vae-oid.npz'

_SEGMENT_DETECT_RE = re.compile(
    r'(.*?)' +
    r'<loc(\d{4})>' * 4 + r'\s*' +
    '(?:%s)?' % (r'<seg(\d{3})>' * 16) +
    r'\s*([^;<>]+)? ?(?:; )?',
)


def _get_params(checkpoint):

  def transp(kernel):
    return np.transpose(kernel, (2, 3, 1, 0))

  def conv(name):
    return {
        'bias': checkpoint[name + '.bias'],
        'kernel': transp(checkpoint[name + '.weight']),
    }

  def resblock(name):
    return {
        'Conv_0': conv(name + '.0'),
        'Conv_1': conv(name + '.2'),
        'Conv_2': conv(name + '.4'),
    }

  return {
      '_embeddings': checkpoint['_vq_vae._embedding'],
      'Conv_0': conv('decoder.0'),
      'ResBlock_0': resblock('decoder.2.net'),
      'ResBlock_1': resblock('decoder.3.net'),
      'ConvTranspose_0': conv('decoder.4'),
      'ConvTranspose_1': conv('decoder.6'),
      'ConvTranspose_2': conv('decoder.8'),
      'ConvTranspose_3': conv('decoder.10'),
      'Conv_1': conv('decoder.12'),
  }


def _quantized_values_from_codebook_indices(codebook_indices, embeddings):
  batch_size, num_tokens = codebook_indices.shape
  assert num_tokens == 16, codebook_indices.shape
  unused_num_embeddings, embedding_dim = embeddings.shape

  encodings = jnp.take(embeddings, codebook_indices.reshape((-1)), axis=0)
  encodings = encodings.reshape((batch_size, 4, 4, embedding_dim))
  return encodings


@functools.cache
def _get_reconstruct_masks():
  """Reconstructs masks from codebook indices.
  Returns:
    A function that expects indices shaped `[B, 16]` of dtype int32, each
    ranging from 0 to 127 (inclusive), and that returns a decoded masks sized
    `[B, 64, 64, 1]`, of dtype float32, in range [-1, 1].
  """

  class ResBlock(nn.Module):
    features: int

    @nn.compact
    def __call__(self, x):
      original_x = x
      x = nn.Conv(features=self.features, kernel_size=(3, 3), padding=1)(x)
      x = nn.relu(x)
      x = nn.Conv(features=self.features, kernel_size=(3, 3), padding=1)(x)
      x = nn.relu(x)
      x = nn.Conv(features=self.features, kernel_size=(1, 1), padding=0)(x)
      return x + original_x

  class Decoder(nn.Module):
    """Upscales quantized vectors to mask."""

    @nn.compact
    def __call__(self, x):
      num_res_blocks = 2
      dim = 128
      num_upsample_layers = 4

      x = nn.Conv(features=dim, kernel_size=(1, 1), padding=0)(x)
      x = nn.relu(x)

      for _ in range(num_res_blocks):
        x = ResBlock(features=dim)(x)

      for _ in range(num_upsample_layers):
        x = nn.ConvTranspose(
            features=dim,
            kernel_size=(4, 4),
            strides=(2, 2),
            padding=2,
            transpose_kernel=True,
        )(x)
        x = nn.relu(x)
        dim //= 2

      x = nn.Conv(features=1, kernel_size=(1, 1), padding=0)(x)

      return x

  def reconstruct_masks(codebook_indices):
    quantized = _quantized_values_from_codebook_indices(
        codebook_indices, params['_embeddings']
    )
    return Decoder().apply({'params': params}, quantized)

  with open(_MODEL_PATH, 'rb') as f:
    params = _get_params(dict(np.load(f)))

  return jax.jit(reconstruct_masks, backend='cpu')

def extract_objs(text, width, height, unique_labels=False):
  """Returns objs for a string with "<loc>" and "<seg>" tokens."""
  objs = []
  seen = set()
  while text:
    m = _SEGMENT_DETECT_RE.match(text)
    if not m:
      break
    gs = list(m.groups())
    before = gs.pop(0)
    name = gs.pop()
    y1, x1, y2, x2 = [int(x) / 1024 for x in gs[:4]]

    y1, x1, y2, x2 = map(round, (y1*height, x1*width, y2*height, x2*width))
    seg_indices = gs[4:20]
    if seg_indices[0] is None:
      mask = None
    else:
      seg_indices = np.array([int(x) for x in seg_indices], dtype=np.int32)
      m64, = _get_reconstruct_masks()(seg_indices[None])[..., 0]
      m64 = np.clip(np.array(m64) * 0.5 + 0.5, 0, 1)
      m64 = PIL.Image.fromarray((m64 * 255).astype('uint8'))
      mask = np.zeros([height, width])
      if y2 > y1 and x2 > x1:
        mask[y1:y2, x1:x2] = np.array(m64.resize([x2 - x1, y2 - y1])) / 255.0

    content = m.group()
    if before:
      objs.append(dict(content=before))
      content = content[len(before):]
    while unique_labels and name in seen:
      name = (name or '') + "'"
    seen.add(name)
    objs.append(dict(
        content=content, xyxy=(x1, y1, x2, y2), mask=mask, name=name))
    text = text[len(before) + len(content):]

  if text:
    objs.append(dict(content=text))

  return objs

In [ ]:
result = result[len(input_text):].lstrip("\n")

In [ ]:
width = input_image.size[0]
height = input_image.size[1]
masks = extract_objs(result, width, height)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(input_image, cmap='gray')

for _ in masks:
    plt.imshow(_['mask'], cmap='viridis', alpha=0.4, vmin=0, vmax=1)

plt.axis('off')
plt.show()